In [16]:
import sys
print(sys.version)

3.7.1 (default, Dec 14 2018, 13:28:58) 
[Clang 4.0.1 (tags/RELEASE_401/final)]


In [1]:
import numpy as np
import pandas as pd
import pickle
import gzip

# Avoid restarting Kernel
%load_ext autoreload
%autoreload 2

%autosave 50

Autosaving every 50 seconds


## Get user comments

In [2]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path, trunc=0):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1        
    if trunc > 0 and i > trunc: 
        break
  return pd.DataFrame.from_dict(df, orient='index')

In [3]:
data_path = '../../datasets/'
file_name = 'reviews_Movies_and_TV.json.gz'

In [4]:
# comments_df = getDF(data_path + file_name, 200000)
# comments_df.loc[0,'reviewText']
# print(comments_df.shape)
# comments_df.head(1)

In [5]:
# Save the records
# import pickle
# pickle_out = open(data_path + "amzn_200k.pickle","wb")
# pickle.dump(comments_df, pickle_out)
# pickle_out.close()

In [6]:
# Reads records
import pickle
pickle_in = open(data_path + "amzn_200k.pickle","rb")
comments_df = pickle.load(pickle_in)

## Split comments into separate sentences

In [61]:
small = comments_df.loc[:1, :]
len(small.index)

2

In [62]:
from nltk.tokenize import sent_tokenize
small['sentence'] = small['reviewText'].map(sent_tokenize)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [71]:
small.columns

Index(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText',
       'overall', 'summary', 'unixReviewTime', 'reviewTime', 'sentence'],
      dtype='object')

In [ ]:
small.drop(['reviewerName', 'helpful', 'reviewText', 'summary', 
            'unixReviewTime', 'reviewTime'], axis=1, inplace=True)

In [76]:
small

,reviewerID,asin,overall,sentence
0,A3R5OBKS7OM2IR,0000143502,5.0,[This has some great tips as always and is hel...
1,A3R5OBKS7OM2IR,0000143529,5.0,"[This is a great pastry guide., I love how Alt..."


In [77]:
sentences = small['sentence'] \
.apply(pd.Series) \
.merge(small, left_index = True, right_index = True) \
.drop(['sentence'], axis = 1) \
.melt(id_vars = ['reviewerID', 'asin','overall'], value_name = 'sentence') \
.drop(['variable'], axis = 1) \
.dropna()

print(sentences.shape)
sentences.head(30)

(6, 4)


,reviewerID,asin,overall,sentence
0,A3R5OBKS7OM2IR,0000143502,5.0,This has some great tips as always and is help...
1,A3R5OBKS7OM2IR,0000143529,5.0,This is a great pastry guide.
2,A3R5OBKS7OM2IR,0000143502,5.0,"I haven't tried any of the recipes yet, but I ..."
3,A3R5OBKS7OM2IR,0000143529,5.0,I love how Alton's collections can break it do...
4,A3R5OBKS7OM2IR,0000143502,5.0,Sometimes it's just lovely to let Alton entert...
5,A3R5OBKS7OM2IR,0000143529,5.0,I might even try some of these recipes some day.


## Sentence-level prep & cleaning

### Split into words

In [78]:
from nltk.tokenize import word_tokenize

sentences['words'] = sentences['sentence'].apply(lambda s: word_tokenize(s))

### Remove support-related sentences

### Remove objective sentences for case B

In [86]:
sentences

,reviewerID,asin,overall,sentence,words
0,A3R5OBKS7OM2IR,0000143502,5.0,This has some great tips as always and is help...,"[This, has, some, great, tips, as, always, and..."
1,A3R5OBKS7OM2IR,0000143529,5.0,This is a great pastry guide.,"[This, is, a, great, pastry, guide, .]"
2,A3R5OBKS7OM2IR,0000143502,5.0,"I haven't tried any of the recipes yet, but I ...","[I, have, n't, tried, any, of, the, recipes, y..."
3,A3R5OBKS7OM2IR,0000143529,5.0,I love how Alton's collections can break it do...,"[I, love, how, Alton, 's, collections, can, br..."
4,A3R5OBKS7OM2IR,0000143502,5.0,Sometimes it's just lovely to let Alton entert...,"[Sometimes, it, 's, just, lovely, to, let, Alt..."
5,A3R5OBKS7OM2IR,0000143529,5.0,I might even try some of these recipes some day.,"[I, might, even, try, some, of, these, recipes..."


### Numpy approach

In [80]:
sent_arr = sentences.to_numpy()
sent_arr[0]

array(['A3R5OBKS7OM2IR', '0000143502', 5.0,
       'This has some great tips as always and is helping me to complete my Good Eats collection.'],
      dtype=object)

In [81]:
sent_arr[:, 3]

array(['This has some great tips as always and is helping me to complete my Good Eats collection.',
       'This is a great pastry guide.',
       "I haven't tried any of the recipes yet, but I will soon.",
       "I love how Alton's collections can break it down so baking isn't so mystical and scary.",
       "Sometimes it's just lovely to let Alton entertain us.",
       'I might even try some of these recipes some day.'], dtype=object)

In [82]:
# words = [sent for sent in map(word_tokenize, sent_arr[0, 3])]
# words

In [83]:
# sentences['words'] = 
# sentences.apply(lambda row: print(type(row['sentence'])),axis=1) #
# sentences.apply(lambda row: ['one','two'],axis=1) #
sentences.apply(lambda row: word_tokenize(row['sentence']), axis=1)

0    [This, has, some, great, tips, as, always, and...
1               [This, is, a, great, pastry, guide, .]
2    [I, have, n't, tried, any, of, the, recipes, y...
3    [I, love, how, Alton, 's, collections, can, br...
4    [Sometimes, it, 's, just, lovely, to, let, Alt...
5    [I, might, even, try, some, of, these, recipes...
dtype: object

In [56]:
paragraph = "My mother drove me to the airport with the windows rolled down. It was seventy-five degrees in Phoenix, the sky a perfect, cloudless blue. I was wearing my favorite shirt – sleeveless, white eyelet lace; I was wearing it as a farewell gesture. My carry-on item was a parka. In the Olympic Peninsula of northwest Washington State, a small town named Forks exists under a near-constant cover of clouds. It rains on this inconsequential town more than any other place in the United States of America. It was from this town and its gloomy, omnipresent shade that my mother escaped with me when I was only a few months old. It was in this town that I’d been compelled to spend a month every summer until I was fourteen. That was the year I finally put my foot down; these past three summers, my dad, Charlie, vacationed with me in California for two weeks instead."
paragraph
import unicodedata

def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return only_ascii.decode()

input_string = remove_accents(paragraph)
from nltk.tokenize import sent_tokenize

sent_tokens = np.array(sent_tokenize(input_string))

In [57]:
sent_tokens

array(['My mother drove me to the airport with the windows rolled down.',
       'It was seventy-five degrees in Phoenix, the sky a perfect, cloudless blue.',
       'I was wearing my favorite shirt  sleeveless, white eyelet lace; I was wearing it as a farewell gesture.',
       'My carry-on item was a parka.',
       'In the Olympic Peninsula of northwest Washington State, a small town named Forks exists under a near-constant cover of clouds.',
       'It rains on this inconsequential town more than any other place in the United States of America.',
       'It was from this town and its gloomy, omnipresent shade that my mother escaped with me when I was only a few months old.',
       'It was in this town that Id been compelled to spend a month every summer until I was fourteen.',
       'That was the year I finally put my foot down; these past three summers, my dad, Charlie, vacationed with me in California for two weeks instead.'],
      dtype='<U144')

In [58]:
from nltk.tokenize import word_tokenize

tokens = [sent for sent in map(word_tokenize, sent_tokens)]
tokens

[['My',
  'mother',
  'drove',
  'me',
  'to',
  'the',
  'airport',
  'with',
  'the',
  'windows',
  'rolled',
  'down',
  '.'],
 ['It',
  'was',
  'seventy-five',
  'degrees',
  'in',
  'Phoenix',
  ',',
  'the',
  'sky',
  'a',
  'perfect',
  ',',
  'cloudless',
  'blue',
  '.'],
 ['I',
  'was',
  'wearing',
  'my',
  'favorite',
  'shirt',
  'sleeveless',
  ',',
  'white',
  'eyelet',
  'lace',
  ';',
  'I',
  'was',
  'wearing',
  'it',
  'as',
  'a',
  'farewell',
  'gesture',
  '.'],
 ['My', 'carry-on', 'item', 'was', 'a', 'parka', '.'],
 ['In',
  'the',
  'Olympic',
  'Peninsula',
  'of',
  'northwest',
  'Washington',
  'State',
  ',',
  'a',
  'small',
  'town',
  'named',
  'Forks',
  'exists',
  'under',
  'a',
  'near-constant',
  'cover',
  'of',
  'clouds',
  '.'],
 ['It',
  'rains',
  'on',
  'this',
  'inconsequential',
  'town',
  'more',
  'than',
  'any',
  'other',
  'place',
  'in',
  'the',
  'United',
  'States',
  'of',
  'America',
  '.'],
 ['It',
  'was',
  

## Create emotion vectors

In [54]:
small = comments_df.loc[:2000, :]
len(small.index)

2001

In [ ]:
%reload_ext autoreload
from emotions_seven import Emotions7
emote = Emotions7()

In [ ]:
comments_emotions = emote.vectorize(small,'reviewText')

In [ ]:
comments_emotions.shape